In [8]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import numpy as np
import pandas as pd
scope = ["https://spreadsheets.google.com/feeds", 
         "https://www.googleapis.com/auth/drive"]


creds = ServiceAccountCredentials.from_json_keyfile_name('projekty-434020-04c49456444d.json', scopes=scope)
client = gspread.authorize(creds)
sheet_id = '1Hx82htjYfapGa4MA1w7mgzGpmEe4myqER3MlYLXfbv8'
sheet = client.open_by_key(sheet_id)


databaza = sheet.get_worksheet(1)
data = databaza.get_all_values()

# Convert the data into a DataFrame
df = pd.DataFrame(data)
df = pd.DataFrame(data[1:], columns=data[0])

In [9]:
df.head()

,Nazov_turnaj,Zapasnik1,Zapasnik2,NO_CONTEST,DRAW,W,L,Kolo,Čas,Metoda,Disciplina,Vaha,CANCELLED,Zapas_typ
0,Clash of the Stars 1,"['Leo Brichta','Lukáš Fajk']","['Daniel Šutera','Michal Zbožínek']",0,0,"['Leo Brichta','Lukáš Fajk']","['Daniel Šutera','Michal Zbožínek']",1,,TKO,MMA,,0,Main card
1,Clash of the Stars 1,"Lukáš ""Luktuma"" Tůma","Tadeáš ""Gangstaboy"" Veselý",0,0,"Lukáš ""Luktuma"" Tůma","Tadeáš ""Gangstaboy"" Veselý",1,,TKO,MMA,,0,Main card
2,Clash of the Stars 1,"Martin ""Budík"" Budín","Lukáš ""Huhu"" Procházka",0,0,"Martin ""Budík"" Budín","Lukáš ""Huhu"" Procházka",1,,TKO,Box,,0,Main card
3,Clash of the Stars 1,Filip Grznár,"Michal ""Psycho Michal"" Peštuka",0,0,Filip Grznár,"Michal ""Psycho Michal"" Peštuka",1,,TKO,MMA,,0,Main card
4,Clash of the Stars 1,"Vít ""Vítek z FIZIstyle"" Daňek","Daniel ""Danny z TVTwixx"" Sonnleitner",0,0,"Vít ""Vítek z FIZIstyle"" Daňek","Daniel ""Danny z TVTwixx"" Sonnleitner",1,,SUB,MMA,,0,Main card


In [10]:
import pandas as pd

# Create an empty DataFrame for fighters' statistics
fighters_stats = pd.DataFrame(columns=["Meno", "Priezvisko", "Prezývka", "W", "L", "D", "NO_CONTEST", 
                                       "CANCEL", "W_OTHER", "L_OTHER", "W_KO/TKO", "W_SUB", "W_DEC", 
                                       "L_KO/TKO", "L_SUB", "L_DEC"])

# Function to split name into Meno, Priezvisko, and Prezývka
def split_name(fighter_name):
    if '"' in fighter_name:
        name_parts = fighter_name.split('"')
        prezyvka = name_parts[1].strip()
        full_name = name_parts[0].strip() + " " + name_parts[2].strip()
    else:
        full_name = fighter_name.strip()
        prezyvka = ""
    
    name_parts = full_name.split()
    meno = name_parts[0]
    priezvisko = ' '.join(name_parts[1:])
    
    return meno, priezvisko, prezyvka

# Function to update or create a fighter's record
def update_fighter_stats(fighter_name, method, outcome, is_disqualification=False):
    meno, priezvisko, prezyvka = split_name(fighter_name)

    # Check if the fighter already exists in the DataFrame
    fighter_row = fighters_stats[(fighters_stats["Meno"] == meno) & (fighters_stats["Priezvisko"] == priezvisko)]
    
    if fighter_row.empty:
        # Create a new record if not exists
        new_row = {
            "Meno": meno, "Priezvisko": priezvisko, "Prezývka": prezyvka, 
            "W": 0, "L": 0, "D": 0, "NO_CONTEST": 0, "CANCEL": 0, 
            "W_OTHER": 0, "L_OTHER": 0, "W_KO/TKO": 0, "W_SUB": 0, "W_DEC": 0, 
            "L_KO/TKO": 0, "L_SUB": 0, "L_DEC": 0
        }
        fighters_stats.loc[len(fighters_stats)] = new_row
        fighter_row = fighters_stats[(fighters_stats["Meno"] == meno) & (fighters_stats["Priezvisko"] == priezvisko)]
    
    index = fighter_row.index[0]

    # Update statistics based on the outcome and method
    if outcome == "W":
        fighters_stats.loc[index, "W"] += 1
        if method == "KO":
            fighters_stats.loc[index, "W_KO/TKO"] += 1
        elif method == "TKO":
            fighters_stats.loc[index, "W_KO/TKO"] += 1
        elif method == "SUB":
            fighters_stats.loc[index, "W_SUB"] += 1
        elif method == "DEC":
            fighters_stats.loc[index, "W_DEC"] += 1
        if is_disqualification:
            fighters_stats.loc[index, "W_OTHER"] += 1
    elif outcome == "L":
        fighters_stats.loc[index, "L"] += 1
        if method == "KO":
            fighters_stats.loc[index, "L_KO/TKO"] += 1
        elif method == "TKO":
            fighters_stats.loc[index, "L_KO/TKO"] += 1
        elif method == "SUB":
            fighters_stats.loc[index, "L_SUB"] += 1
        elif method == "DEC":
            fighters_stats.loc[index, "L_DEC"] += 1
        if is_disqualification:
            fighters_stats.loc[index, "L_OTHER"] += 1
    elif outcome == "D":
        fighters_stats.loc[index, "D"] += 1
    
    elif outcome =='CANCEL':
        fighters_stats.loc[index, "CANCEL"] += 1
    
    elif outcome =='NO_CONTEST':
        fighters_stats.loc[index, "NO_CONTEST"] += 1

# Function to parse the list of fighters and update stats
def parse_fighters_and_update_stats(fighters, method, outcome, is_disqualification=False):
    # Clean the string and split into individual fighters
    fighters_list = fighters.strip("[]").replace("'", "").split(",")
    for fighter in fighters_list:
        update_fighter_stats(fighter.strip(), method, outcome, is_disqualification)

# Now loop through each row of your dataset
for _, row in df.iterrows():
    if row["CANCELLED"] == '1':
        # Update CANCEL for both fighters
        parse_fighters_and_update_stats(row["Zapasnik1"], "", "CANCEL")
        parse_fighters_and_update_stats(row["Zapasnik2"], "", "CANCEL")
        continue

    if row["NO_CONTEST"] == '1':
        # Update NO_CONTEST for both fighters
        parse_fighters_and_update_stats(row["Zapasnik1"], "", "NO_CONTEST")
        parse_fighters_and_update_stats(row["Zapasnik2"], "", "NO_CONTEST")
        continue

    # Handle Draw
    if row["DRAW"] == '1':
        parse_fighters_and_update_stats(row["Zapasnik1"], "", "D")
        parse_fighters_and_update_stats(row["Zapasnik2"], "", "D")
        continue
    
    # Winners and Losers
    winners = row["W"]
    losers = row["L"]
    
    # Determine the method
    method = row["Metoda"]
    is_disqualification = "DIS" in method

    # Update stats for each winner
    parse_fighters_and_update_stats(winners, method, "W", is_disqualification)
    
    # Update stats for each loser
    parse_fighters_and_update_stats(losers, method, "L", is_disqualification)

# Output the final DataFrame
fighters_stats


,Meno,Priezvisko,Prezývka,W,L,D,NO_CONTEST,CANCEL,W_OTHER,L_OTHER,W_KO/TKO,W_SUB,W_DEC,L_KO/TKO,L_SUB,L_DEC
0,Leo,Brichta,,1,0,0,0,0,0,0,1,0,0,0,0,0
1,Lukáš,Fajk,,1,0,0,0,0,0,0,1,0,0,0,0,0
2,Daniel,Šutera,,0,1,0,0,0,0,0,0,0,0,1,0,0
3,Michal,Zbožínek,,0,1,0,0,0,0,0,0,0,0,1,0,0
4,Lukáš,Tůma,Luktuma,2,1,0,0,0,0,0,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,Josef,Škop,Tyson,1,0,0,0,0,0,0,1,0,0,0,0,0
132,Hynek,Hospodárský,Shelby,0,1,0,0,0,0,0,0,0,0,1,0,0
133,Dan,Suchomel,Opat04,0,1,0,0,0,0,0,0,0,0,1,0,0
134,Pavel,Novotný,Král Bulváru,1,0,0,0,0,0,0,1,0,0,0,0,0


In [11]:
fighters_stats[fighters_stats['Prezývka'] =='Kukisman']

,Meno,Priezvisko,Prezývka,W,L,D,NO_CONTEST,CANCEL,W_OTHER,L_OTHER,W_KO/TKO,W_SUB,W_DEC,L_KO/TKO,L_SUB,L_DEC
67,Ladislav,Gal,Kukisman,3,0,0,0,0,0,0,2,0,1,0,0,0


In [12]:
# Replace NaN, inf, and -inf values as before
fighters_stats = fighters_stats.replace({np.nan: '', np.inf: 1e10, -np.inf: -1e10})

# Convert all 0s to strings to prevent Google Sheets from formatting them as time
fighters_stats = fighters_stats.applymap(lambda x: '0' if x == 0 else x)

# Prepare the data for updating the sheet
df_list = [fighters_stats.columns.values.tolist()] + fighters_stats.values.tolist()

# Update the Google Sheet
databaza = sheet.get_worksheet(2)
databaza.clear()
databaza.update(df_list)

C:\Users\admin\AppData\Local\Temp\ipykernel_15248\1148051158.py:13: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  databaza.update(df_list)


{'spreadsheetId': '1Hx82htjYfapGa4MA1w7mgzGpmEe4myqER3MlYLXfbv8',
 'updatedRange': 'Fighters!A1:P137',
 'updatedRows': 137,
 'updatedColumns': 16,
 'updatedCells': 2192}

### TESTY NAJDENIE FIGHTERA V LISTE

In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import numpy as np
import pandas as pd
scope = ["https://spreadsheets.google.com/feeds", 
         "https://www.googleapis.com/auth/drive"]


creds = ServiceAccountCredentials.from_json_keyfile_name('random-project-420514-74ed8d01ffb1.json', scopes=scope)
client = gspread.authorize(creds)
sheet_id = '1Hx82htjYfapGa4MA1w7mgzGpmEe4myqER3MlYLXfbv8'
sheet = client.open_by_key(sheet_id)

databaza = sheet.get_worksheet(1)
data = databaza.get_all_values()

# Convert the data into a DataFrame
df = pd.DataFrame(data)
df = pd.DataFrame(data[1:], columns=data[0])

In [16]:

df['W'].str.contains('Karel "Karlos" Benda', na=False).sum()

4